In [1]:
!pip install datatable

Active code page: 1252


In [2]:
import datatable as dt
import pandas as pd
import numpy as np
import evalml
df = pd.read_csv('data_eval.csv')

In [3]:
df.head()

,Order Date,Order ID,Order Quantity,Product Container,Product Name,Product Sub-Category,Sales,Ship Mode
0,1/27/2007,24544,31,Medium Box,Canon MP41DH Printing Calculator,Office Machines,6567.0,Express Air
1,1/27/2007,24544,39,Large Box,Fellowes Neat Ideas® Storage Cubes,Storage & Organization,1780.0,Regular Air
2,1/27/2007,24544,15,Jumbo Drum,"Global Stack Chair without Arms, Black",Chairs & Chairmats,578.0,Delivery Truck
3,1/27/2007,20422,30,Small Pack,Nu-Dell Leatherette Frames,Office Furnishings,611.0,Regular Air
4,1/27/2007,55937,10,Small Box,"IBM Active Response Keyboard, Black",Computer Peripherals,517.0,Regular Air


In [4]:
#X=DT[:,0:7]
#y= DT[:,7]
X = df.drop('Ship Mode',axis=1)
y = df['Ship Mode']
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='Multiclass')

In [5]:
X_train.shape

(6282, 7)

In [6]:
X_train.head()

,Order Date,Order ID,Order Quantity,Product Container,Product Name,Product Sub-Category,Sales
1682,2007-11-24,48295,48,Medium Box,"Soundgear Copyboard Conference Phone, Optional...",Office Machines,11836.0
5601,2009-11-18,36640,18,Small Pack,"Verbatim DVD-R, 3.95GB, SR, Mitsubishi Branded...",Computer Peripherals,884.0
3249,2008-10-09,18213,29,Small Box,"White Dual Perf Computer Printout Paper, 2700 ...",Paper,1955.0
5490,2009-10-28,32001,22,Small Pack,"Executive Impressions 14"" Contract Wall Clock",Office Furnishings,803.0
2655,2008-05-23,13540,33,Small Box,Avery 510,Labels,166.0


### Running the AutoML to select the best algorithm

In [7]:
import evalml
evalml.problem_types.ProblemTypes.all_problem_types

[<ProblemTypes.BINARY: 'binary'>,
 <ProblemTypes.MULTICLASS: 'multiclass'>,
 <ProblemTypes.REGRESSION: 'regression'>,
 <ProblemTypes.TIME_SERIES_REGRESSION: 'time series regression'>,
 <ProblemTypes.TIME_SERIES_BINARY: 'time series binary'>,
 <ProblemTypes.TIME_SERIES_MULTICLASS: 'time series multiclass'>]

In [8]:
from evalml.automl import AutoMLSearch
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='multiclass')
automl.search()

In [9]:
automl.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,8,Elastic Net Classifier w/ Label Encoder + Sele...,8,0.359508,0.000637,0.359508,95.879764,False,"{'Label Encoder': {'positive_label': None}, 'N..."
1,3,Logistic Regression Classifier w/ Label Encode...,3,0.364749,0.003214,0.364749,95.819694,False,"{'Label Encoder': {'positive_label': None}, 'D..."
2,1,Logistic Regression Classifier w/ Label Encode...,1,0.367544,0.003853,0.367544,95.787658,False,"{'Label Encoder': {'positive_label': None}, 'D..."
3,10,XGBoost Classifier w/ Label Encoder + Select C...,10,0.373038,0.004350,0.373038,95.724696,False,"{'Label Encoder': {'positive_label': None}, 'N..."
4,4,Random Forest Classifier w/ Label Encoder + Da...,4,0.382049,0.002323,0.382049,95.621419,False,"{'Label Encoder': {'positive_label': None}, 'D..."
5,6,LightGBM Classifier w/ Label Encoder + Select ...,6,0.390794,0.001931,0.390794,95.521194,False,"{'Label Encoder': {'positive_label': None}, 'N..."
6,7,Extra Trees Classifier w/ Label Encoder + Sele...,7,0.408191,0.001900,0.408191,95.321817,False,"{'Label Encoder': {'positive_label': None}, 'N..."
7,2,Random Forest Classifier w/ Label Encoder + Da...,2,0.414736,0.001350,0.414736,95.246802,False,"{'Label Encoder': {'positive_label': None}, 'D..."
8,5,Decision Tree Classifier w/ Label Encoder + Se...,5,0.431632,0.072451,0.431632,95.053162,False,"{'Label Encoder': {'positive_label': None}, 'N..."
9,9,CatBoost Classifier w/ Label Encoder + Select ...,9,0.824509,0.001276,0.824509,90.550490,False,"{'Label Encoder': {'positive_label': None}, 'N..."


### Best pipeline

In [10]:
automl.best_pipeline

pipeline = MulticlassClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Numeric Pipeline - Select Columns By Type Transformer': ['Select Columns By Type Transformer', 'X', 'Label Encoder.y'], 'Numeric Pipeline - Label Encoder': ['Label Encoder', 'Numeric Pipeline - Select Columns By Type Transformer.x', 'Label Encoder.y'], 'Numeric Pipeline - DateTime Featurizer': ['DateTime Featurizer', 'Numeric Pipeline - Select Columns By Type Transformer.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - Imputer': ['Imputer', 'Numeric Pipeline - DateTime Featurizer.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - One Hot Encoder': ['One Hot Encoder', 'Numeric Pipeline - Imputer.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - Oversampler': ['Oversampler', 'Numeric Pipeline - One Hot Encoder.x', 'Numeric Pipeline - Label Encoder.y'], 'Numeric Pipeline - Standard Scaler': ['Standard Scaler', 'Numeric Pipeline - Oversampler.x'

In [11]:
best_pipeline=automl.best_pipeline

### Check the best discription

In [12]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])


************************************************************************************************************************************************************************************************************************************************************************************************************************
* Elastic Net Classifier w/ Label Encoder + Select Columns By Type Transformer + Label Encoder + DateTime Featurizer + Imputer + One Hot Encoder + Oversampler + Standard Scaler + Select Columns Transformer + Select Columns Transformer + Label Encoder + Imputer + One Hot Encoder + Oversampler + Standard Scaler *
************************************************************************************************************************************************************************************************************************************************************************************************************************

Problem Type: multiclass
Model Family: Linear

Pipeline Ste

In [13]:
### Evaluate on hold out data
best_pipeline.score(X_test, y_test, objectives=["f1 Weighted", "Precision Weighted","Recall Weighted"])

OrderedDict([('F1 Weighted', 0.8285713711092542),
             ('Precision Weighted', 0.7816236668107235),
             ('Recall Weighted', 0.8828771483131763)])

### Optimize

In [14]:
automl_auc = AutoMLSearch(X_train=X_train, y_train=y_train,
                          problem_type='multiclass',
                          objective='f1 Weighted',
                          additional_objectives=['f1 Weighted', 'precision Weighted'],
                          max_batches=1,
                          optimize_thresholds=True)

automl_auc.search()

In [15]:
automl_auc.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,1,Logistic Regression Classifier w/ Label Encode...,1,0.828990,0.002252,0.828990,18.966785,False,"{'Label Encoder': {'positive_label': None}, 'D..."
1,2,Random Forest Classifier w/ Label Encoder + Da...,2,0.828290,0.000394,0.828290,18.896811,False,"{'Label Encoder': {'positive_label': None}, 'D..."
2,0,Mode Baseline Multiclass Classification Pipeline,0,0.639322,0.000000,0.639322,0.000000,False,"{'Label Encoder': {'positive_label': None}, 'B..."


In [16]:
automl_auc.describe_pipeline(automl_auc.rankings.iloc[0]["id"])


*************************************************************************************************************************************
* Logistic Regression Classifier w/ Label Encoder + DateTime Featurizer + Imputer + One Hot Encoder + Oversampler + Standard Scaler *
*************************************************************************************************************************************

Problem Type: multiclass
Model Family: Linear

Pipeline Steps
1. Label Encoder
	 * positive_label : None
2. DateTime Featurizer
	 * features_to_extract : ['year', 'month', 'day_of_week', 'hour']
	 * encode_as_categories : False
	 * time_index : None
3. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
4. One Hot Encoder
	 * top_n : 10
	 * features_to_encode : None
	 * categories : None
	 * drop : if_binary
	 * handle_unknown : ignore
	 * handle_missing : error
5. Oversampler
	 * samplin

In [17]:
best_pipeline_auc = automl_auc.best_pipeline

In [18]:
# get the score on holdout data
best_pipeline_auc.score(X_test, y_test,  objectives=["F1 Weighted", "Precision Weighted","Recall Weighted"])

OrderedDict([('F1 Weighted', 0.8294641120645261),
             ('Precision Weighted', 0.820991948939222),
             ('Recall Weighted', 0.8822406110757479)])

In [19]:
best_pipeline.save("EvalMlmodel.pkl")

### Load the model

In [20]:
check_model=automl.load('EvalMlmodel.pkl')

In [21]:
check_model.predict_proba(X_test)

,Delivery Truck,Express Air,Regular Air
3789,0.999559,0.000162,0.000278
586,0.000219,0.293121,0.706661
459,0.000233,0.183133,0.816634
969,0.000262,0.188498,0.811240
488,0.999723,0.000160,0.000117
...,...,...,...
1959,0.000396,0.186000,0.813604
977,0.000120,0.306067,0.693812
5775,0.000203,0.161295,0.838502
3616,0.000127,0.218569,0.781305
